## Summary 

- DATA USED - Naviga_Entity_Extraction_v02.xlsx


- FILES CREATED - location_norm2raw_v1, location_raw2norm_v1, company_norm2raw_v1, company_raw2norm_v1, persons_norm2raw_v1, persons_raw2norm_v1
               

- Matching algorithms of notebook 'entity_normalization_ver_0_3_6' were combined in a script file Normalisation_Helper.py and used, Co Document Ratio was not used. DKE-Intervention/Post-processing was done for all 3 types of entities manually.



In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import json
import math
import copy
import random
from collections import defaultdict, Counter, OrderedDict
from itertools import combinations
import unicodedata
import re
import matplotlib.pyplot as plt
import pickle

In [2]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import Levenshtein as lvst
from metaphone import doublemetaphone as dmt_phone
import networkx as nx
import nxviz as nv

In [4]:
from sparse_dot_topn import awesome_cossim_topn as cossim_topn

In [5]:
from Normalisation_Helper import get_df_match,add_match_score,cut_off,get_connected_comp,get_postProcessigUnits
from Normalisation_Helper import choose_normalized,get_comp_final,make_norm_dict,normalize_unicode_to_ascii

In [7]:
df=pd.read_excel("Naviga5_Entity_Extraction_v02.xlsx")
df = df.fillna("")

In [8]:
persons= df["Person"]
company = df["Company"]
location = df["Location"]

## Persons


In [9]:
df_persons = get_df_match(persons, 'Person', wiki_match=False, sorting_1st_letters=True)

# of candidate pairs: 32969


In [10]:
weights_person = [4,2,1,1,1]
add_match_score(df_persons, weights=weights_person)

In [11]:
df_persons = cut_off(df_persons)

In [12]:
df_persons.shape

(1952, 8)

In [13]:
df_persons.head(10)

,in_text,matched,similarity,first_letter_match,dmtph_match,inv_lvst_dist,jw_sim,match_score
0,DAVID BERNHARDT,David Bernhardt,1.0,True,0.997527,1.0,1.0,0.999725
1,Gretchen Whitmer,GRETCHEN WHITMER,1.0,True,0.997527,1.0,1.0,0.999725
2,LaKeith Stanfield,Lakeith Stanfield,1.0,True,0.997527,1.0,1.0,0.999725
3,Martin Luther King Jr,MARTIN LUTHER KING JR,1.0,True,0.997527,1.0,1.0,0.999725
4,JONATHON N. JOHNSON,Jonathon N. Johnson,1.0,True,0.997527,1.0,1.0,0.999725
5,Martin Luther King Jr,Martin Luther King Jr,1.0,True,0.997527,1.0,1.0,0.999725
6,George H.W. Bush,George H. W. Bush,1.0,True,0.997527,1.0,1.0,0.999725
7,John Doe,JOHN DOE,1.0,True,0.997527,1.0,1.0,0.999725
8,JENNIFER JILL GOOD,Jennifer Jill Good,1.0,True,0.997527,1.0,1.0,0.999725
9,BRENT AGUILAR,Brent Aguilar,1.0,True,0.997527,1.0,1.0,0.999725


In [14]:
df_persons.tail(10)

,in_text,matched,similarity,first_letter_match,dmtph_match,inv_lvst_dist,jw_sim,match_score
1942,Levar Stoney,Levar Stoney's,0.944824,False,1.233946e-04,0.5,1.000000,0.586602
1943,Christian Cooper,Christian Cooper's,0.944672,False,1.233946e-04,0.5,1.000000,0.586535
1944,Anne Miskey,Anne Miskey's,0.944483,False,1.233946e-04,0.5,1.000000,0.586451
1945,Derek Draper's,Derek Draper,0.944419,False,1.233946e-04,0.5,1.000000,0.586422
1946,Dr. Schoifet,Dr. Schoifet's,0.945927,False,1.233946e-04,0.5,0.993939,0.586419
1947,Nancy Pelosi,Nancy Pelosi's,0.944327,False,1.233946e-04,0.5,1.000000,0.586381
1948,Nelson Mandela,Nelson Mandela's,0.944293,False,1.233946e-04,0.5,1.000000,0.586366
1949,David Campbell,David Campbell's,0.944292,False,1.233946e-04,0.5,1.000000,0.586366
1950,Michael Brown,Michael Brown's,0.944235,False,8.315280e-07,0.5,1.000000,0.586327
1951,Ralph Northam,Ralph Northam's,0.944182,False,1.233946e-04,0.5,1.000000,0.586317


In [15]:
persons_connected = get_connected_comp(df_persons)
connected_comp = persons_connected

In [16]:
connected_comp_short, connected_comp_long, connected_comp_all = get_postProcessigUnits(connected_comp)

In [17]:
print(len(connected_comp_short))
print(len(connected_comp_long))
print(len(connected_comp_all))

1731
95
1826


In [18]:
connected_comp_short[:20]

[(0, ['"Bill" Johnson', 'Bill Johnson']),
 (1, ['ABIGAIL KING', 'Abigail Kane']),
 (2, ['ADAM GRIBB', 'Adam Gribb']),
 (3, ['ALBERT A. NATOLI', 'Albert A. Natoli']),
 (4, ["ALEJANDRA O'CONNELL-DOMENECH", "Alejandra O'Connell-Domenech"]),
 (5, ['ARIANA GRANDE', 'Ariana Grande']),
 (6, ['Aaron Bear', 'Aaron Beaver']),
 (7, ['Aaron Green', 'Aaron Greene']),
 (8, ['Aaron Morris', 'AARON MORRISON']),
 (9, ['Aaron Nola', 'Aaron Nolan']),
 (10, ["Aaron Sorkin's", 'Aaron Sorkin']),
 (11, ['Abang Okon', 'Okon Abang']),
 (12, ['Abby Roque', "Abby Roque's"]),
 (13, ['Abdirahim Mohamed Abdullahi', 'Mohamed Abdullahi Abdi']),
 (14, ['Abdur Razzaq', 'Abdul Razzaq']),
 (15, ['Abigail Hardage', 'Abigail Hardy']),
 (16, ['Abigail Jackson', 'Abigail Johnson']),
 (17, ['Abigail Richie', 'Abigail Richardson']),
 (18, ['Abike Dabiri Erewa', 'Abike Dabiri-Erewa']),
 (19, ["Abraham Lincoln's", 'Abraham Lincoln'])]

In [19]:
len(connected_comp_long)

95

In [21]:
connected_comp_all

[(0, ['"Bill" Johnson', 'Bill Johnson']),
 (1, ['ABIGAIL KING', 'Abigail Kane']),
 (2, ['ADAM GRIBB', 'Adam Gribb']),
 (3, ['ALBERT A. NATOLI', 'Albert A. Natoli']),
 (4, ["ALEJANDRA O'CONNELL-DOMENECH", "Alejandra O'Connell-Domenech"]),
 (5, ['ARIANA GRANDE', 'Ariana Grande']),
 (6, ['Aaron Bear', 'Aaron Beaver']),
 (7, ['Aaron Green', 'Aaron Greene']),
 (8, ['Aaron Morris', 'AARON MORRISON']),
 (9, ['Aaron Nola', 'Aaron Nolan']),
 (10, ["Aaron Sorkin's", 'Aaron Sorkin']),
 (11, ['Abang Okon', 'Okon Abang']),
 (12, ['Abby Roque', "Abby Roque's"]),
 (13, ['Abdirahim Mohamed Abdullahi', 'Mohamed Abdullahi Abdi']),
 (14, ['Abdur Razzaq', 'Abdul Razzaq']),
 (15, ['Abigail Hardage', 'Abigail Hardy']),
 (16, ['Abigail Jackson', 'Abigail Johnson']),
 (17, ['Abigail Richie', 'Abigail Richardson']),
 (18, ['Abike Dabiri Erewa', 'Abike Dabiri-Erewa']),
 (19, ["Abraham Lincoln's", 'Abraham Lincoln']),
 (20, ['Adam Lucas', 'Lucas Adams']),
 (21, ['Adam Silverman', 'Adam Silver']),
 (22, ["Adama T

In [22]:
# manual insepction to excude, connect & add
index2exclude = [1,2,8,15,16,17,21,25,32,39,40,44,48,50,51,55,63,72,75,76,80,90,91,92,93,97,101,102,105,106,109,123,153,154,155,156,163,164,171,196,205,206,212,214,242,243,274,281,282,283]
index2connect = [(1,2)]
comp2add = [['Ajit Pai', 'AJIT PAI'],['Anthony Green', 'Anthony Greene'],['Ashley Johns', 'Ashley Johnson','Ashlyn Johnson'],['Chief   Minister', 'Chief Minister', "Chief Minister's"]]

In [23]:
connected_comp_final = get_comp_final(connected_comp_all, connected_comp_short, is_all=True
                                     , index2connect=index2connect
                                     , index2exclude=index2exclude
                                     , comp2add=comp2add
                                     )

In [24]:
print(len(connected_comp))
print(len(connected_comp_final))

1826
1781


In [25]:
persons_connected = connected_comp_final

In [26]:
normalized_terms = choose_normalized(persons_connected, choice='long')

In [27]:
raw2normalized, normalized2raw = make_norm_dict(persons_connected, normalized_terms)

In [28]:
print("# of raw (in-snippet) entiteis normalized: {}".format(len(raw2normalized)))
print("# of normalized keys: {}".format(len(normalized2raw)))

# of raw (in-snippet) entiteis normalized: 3666
# of normalized keys: 1781


In [30]:
ver_string="_v1"
f_name = 'persons_matching' + ver_string
df_persons.to_csv("corpus/" + f_name + ".csv")

In [31]:
persons_raw2norm = raw2normalized
persons_norm2raw = normalized2raw

In [32]:
list(persons_raw2norm.items())[:5]

[('"Bill" Johnson', 'Bill Johnson'),
 ('Bill Johnson', 'Bill Johnson'),
 ('ALBERT A. NATOLI', 'Albert A. Natoli'),
 ('Albert A. Natoli', 'Albert A. Natoli'),
 ("ALEJANDRA O'CONNELL-DOMENECH", "Alejandra O'Connell-Domenech")]

In [33]:
list(persons_norm2raw.items())[:5]

[('Bill Johnson', ['"Bill" Johnson', 'Bill Johnson']),
 ('Albert A. Natoli', ['ALBERT A. NATOLI', 'Albert A. Natoli']),
 ("Alejandra O'Connell-Domenech",
  ["ALEJANDRA O'CONNELL-DOMENECH", "Alejandra O'Connell-Domenech"]),
 ('ARIANA GRANDE', ['ARIANA GRANDE', 'Ariana Grande']),
 ('Aaron Beaver', ['Aaron Bear', 'Aaron Beaver'])]

In [34]:
data_path="corpus/"

In [35]:
f_name = 'persons_raw2norm' + ver_string
json_dp = json.dumps(persons_raw2norm)
with open(data_path + f_name + ".json","w") as f:
    f.write(json_dp)

f_name = 'persons_norm2raw' + ver_string
json_dp = json.dumps(persons_norm2raw)
with open(data_path + f_name + ".json","w") as f:
    f.write(json_dp)

## Company

In [36]:
df_company = get_df_match(company, 'Company', wiki_match=False, sorting_1st_letters=True)

# of candidate pairs: 31157


In [37]:
weights_company = [4,2,1,1,1]
add_match_score(df_company, weights=weights_company)

In [38]:
df_company = cut_off(df_company)

In [39]:
df_company.shape

(2350, 8)

In [40]:
df_company.head(10)

,in_text,matched,similarity,first_letter_match,dmtph_match,inv_lvst_dist,jw_sim,match_score
0,Valdosta State University,VALDOSTA STATE UNIVERSITY,1.0,True,0.997527,1.0,1.0,0.999725
1,Schaeffler Technologies AG &amp; Co. KG,SCHAEFFLER TECHNOLOGIES AG &amp; CO. KG,1.0,True,0.997527,1.0,1.0,0.999725
2,Homeland Security,homeland security,1.0,True,0.997527,1.0,1.0,0.999725
3,Homeland Security,HOMELAND SECURITY,1.0,True,0.997527,1.0,1.0,0.999725
4,SCHAEFFLER TECHNOLOGIES AG &amp; CO. KG,Schaeffler Technologies AG &amp; Co KG,1.0,True,0.997527,1.0,1.0,0.999725
5,homeland security,HOMELAND SECURITY,1.0,True,0.997527,1.0,1.0,0.999725
6,National Fertilizers Limited,NATIONAL FERTILIZERS LIMITED,1.0,True,0.997527,1.0,1.0,0.999725
7,Schaeffler Technologies AG &amp; Co. KG,Schaeffler Technologies AG &amp; Co KG,1.0,True,0.997527,1.0,1.0,0.999725
8,"California State University, Fullerton",California State University Fullerton,1.0,True,0.997527,1.0,1.0,0.999725
9,New York Knicks,"New York, Knicks",1.0,True,0.997527,1.0,1.0,0.999725


In [41]:
df_company.tail(10)

,in_text,matched,similarity,first_letter_match,dmtph_match,inv_lvst_dist,jw_sim,match_score
2340,Mercer School,Mercer's School,0.902415,False,1.233946e-04,0.50,0.991111,0.566766
2341,AtHoc,AtHoc's,0.912028,False,8.315280e-07,0.50,0.952381,0.566721
2342,FTC,FTC's,0.922602,False,1.233946e-04,0.50,0.906667,0.566355
2343,V Sonny Montgomery VA Medical Center,G.V. (Sonny) Montgomery VA Medical Center,0.937078,False,1.233946e-04,0.50,0.848197,0.566293
2344,Hexo's,HEXO,0.915284,False,1.233946e-04,0.50,0.933333,0.566066
2345,NLRB,NLRB's,0.915107,False,1.233946e-04,0.50,0.933333,0.565987
2346,Shenzhen Goodix Technology,SHENZHEN GOODIX TECHNOLOGY CO.,0.960570,False,8.315280e-07,0.25,1.000000,0.565809
2347,Lancet,Lancet's,0.906061,False,1.233946e-04,0.50,0.966667,0.565671
2348,PKK,PKK's,0.920425,False,1.233946e-04,0.50,0.906667,0.565388
2349,St. Paul AME Church,St. Paul's AME Church,0.900523,False,8.315280e-07,0.50,0.985926,0.565336


In [42]:
company_connected = get_connected_comp(df_company)
connected_comp = company_connected

In [43]:
connected_comp_short, connected_comp_long, connected_comp_all = get_postProcessigUnits(connected_comp)

In [49]:
print(len(connected_comp_short))
print(len(connected_comp_long))
print(len(connected_comp_all))

1967
161
2128


In [45]:
connected_comp_short[:20]

[(0, ['29th Infantry Division', '9th Infantry Division']),
 (1, ['3M Company', '3M COMPANY']),
 (2, ["3rd Infantry Division's", '3rd Infantry Division']),
 (3, ['4- H', '4-H']),
 (4, ['A.G.H. Industries, Inc', 'A.G.H. Industries, Inc.']),
 (5, ['ABC News', "ABC News'"]),
 (6, ['ABC-7 Eyewitness News', 'ABC7 Eyewitness News']),
 (7, ['ABS-CBN', "ABS-CBN's"]),
 (8, ['ACE Speedway', 'Ace Speedway']),
 (9, ['AETNA', 'Aetna']),
 (10, ['AFCON', 'Afcon']),
 (11, ['AFRICAN Development Bank', 'African Development Bank']),
 (12,
  ['AGENT Stanisz M J PJM Interconnection, L.L.C.',
   'AUTHOR Stanisz M J PJM Interconnection, L.L.C.']),
 (13, ['AGF', "AGF's"]),
 (14, ['AIR FORCE', 'Air Force']),
 (15, ['ALIBABA GROUP HOLDING LIMITED', 'Alibaba Group Holding Limited']),
 (16, ['AMC Theatres', 'AMC Theater']),
 (17, ['AMF Bowling Centers, Inc.', 'AMF BOWLING CENTERS, INC.']),
 (18, ['ANA Holdings Inc.', 'ANA Holdings Inc']),
 (19, ['APPLIED MATERIALS, INC.', 'Applied Materials Inc.'])]

In [46]:
len(connected_comp_long)

161

In [47]:
connected_comp_long

[(0, ['5N Plus Inc', "5N Plus Inc.'s", '5N Plus Inc.']),
 (1, ['AT&amp;T Inc.', 'AT&amp;T INC.', 'AT&amp;T, Inc.']),
 (2,
  ["African American's",
   'African-American',
   'African American',
   'African- American']),
 (3, ['Amazon', 'AMAZON', 'amazon']),
 (4,
  ['BLACK LIVES MATTER',
   'Black Lives matter',
   'Black Lives Matter',
   'Black. Lives. Matter']),
 (5,
  ['Bethany Lutheran Church',
   'Bethania Lutheran Church',
   'Bethany Lutheran College']),
 (6,
  ['Bill  and  Melinda Gates Foundation',
   'Bill and Melinda Gates Foundation',
   "Bill and Melinda Gates' Foundation"]),
 (7, ['Board of Managers', 'Board of Ambassadors', 'Board of Presidents']),
 (8,
  ['Bureau of Alcohol Tobacco, Firearms, and Explosives',
   'Bureau of Alcohol, Tobacco, Firearms and Explosives',
   'Bureau of Alcohol, Tobacco, Firearms, and Explosives']),
 (9,
  ['Bureau of Investigation &amp; Enforcement',
   "Bureau of Investigation &amp; Enforcement's",
   'Bureau of Investigation and Enforcement'

In [56]:
# manual insepction to excude, connect & add
index2exclude = [2]
index2connect = [(8,9)]
comp2add = ['Department of Defence',
   'Department of Defense']

In [57]:
connected_comp_final = get_comp_final(connected_comp_long, connected_comp_short, is_all=False
                                     , index2connect=index2connect
                                     , index2exclude=index2exclude
                                     , comp2add=comp2add
                                     )

In [58]:
print(len(connected_comp))
print(len(connected_comp_final))

194
194


In [59]:
company_connected = connected_comp_final

In [50]:
normalized_terms = choose_normalized(company_connected, choice='long')

In [51]:
raw2normalized, normalized2raw = make_norm_dict(company_connected, normalized_terms)

In [52]:
print("# of raw (in-snippet) entiteis normalized: {}".format(len(raw2normalized)))
print("# of normalized keys: {}".format(len(normalized2raw)))

# of raw (in-snippet) entiteis normalized: 4439
# of normalized keys: 2128


In [53]:
ver_string="_v1"
f_name = 'company_matching' + ver_string
df_company.to_csv("corpus/" + f_name + ".csv")

In [54]:
company_raw2norm = raw2normalized
company_norm2raw = normalized2raw

In [55]:
list(company_raw2norm.items())[:5]

[('LG Electronics Inc.', 'LG Electronics Inc'),
 ('Lg Electronics Inc.', 'LG Electronics Inc'),
 ('LG Electronics, Inc.', 'LG Electronics Inc'),
 ('LG Electronics Inc', 'LG Electronics Inc'),
 ('LG ELECTRONICS INC.', 'LG Electronics Inc')]

In [56]:
list(company_norm2raw.items())[:5]

[('LG Electronics Inc',
  ['LG Electronics Inc.',
   'Lg Electronics Inc.',
   'LG Electronics, Inc.',
   'LG Electronics Inc',
   'LG ELECTRONICS INC.']),
 ("VMware's", ['Vmware', 'VMWare', "VMware's", 'VMware']),
 ('Office of the General Manager',
  ['Office of the Premier',
   'Office of the General Manager',
   'Office of the Secretary',
   'Office of the Director']),
 ("Cansortium Inc.'s",
  ["Cansortium Inc.'s",
   'Cansortium Inc.',
   'Cansortium Inc',
   'CANSORTIUM INC.']),
 ('Red Cross', ['RED CROSS', 'RED-CROSS', 'RED-Cross', 'Red Cross'])]

In [57]:
data_path="corpus/"

f_name = 'company_raw2norm' + ver_string
json_dp = json.dumps(company_raw2norm)
with open(data_path + f_name + ".json","w") as f:
    f.write(json_dp)

f_name = 'company_norm2raw' + ver_string
json_dp = json.dumps(company_norm2raw)
with open(data_path + f_name + ".json","w") as f:
    f.write(json_dp)

## Location

In [58]:
df_location = get_df_match(location, 'Location', wiki_match=False, sorting_1st_letters=True)

# of candidate pairs: 26020


In [59]:
weights_location = [4,2,1,1,1]
add_match_score(df_location, weights=weights_location)

In [60]:
df_location = cut_off(df_location)

In [61]:
df_location.shape

(3137, 8)

In [62]:
location_connected = get_connected_comp(df_location)
connected_comp = location_connected

In [63]:
df_company.head(10)

,in_text,matched,similarity,first_letter_match,dmtph_match,inv_lvst_dist,jw_sim,match_score
0,Valdosta State University,VALDOSTA STATE UNIVERSITY,1.0,True,0.997527,1.0,1.0,0.999725
1,Schaeffler Technologies AG &amp; Co. KG,SCHAEFFLER TECHNOLOGIES AG &amp; CO. KG,1.0,True,0.997527,1.0,1.0,0.999725
2,Homeland Security,homeland security,1.0,True,0.997527,1.0,1.0,0.999725
3,Homeland Security,HOMELAND SECURITY,1.0,True,0.997527,1.0,1.0,0.999725
4,SCHAEFFLER TECHNOLOGIES AG &amp; CO. KG,Schaeffler Technologies AG &amp; Co KG,1.0,True,0.997527,1.0,1.0,0.999725
5,homeland security,HOMELAND SECURITY,1.0,True,0.997527,1.0,1.0,0.999725
6,National Fertilizers Limited,NATIONAL FERTILIZERS LIMITED,1.0,True,0.997527,1.0,1.0,0.999725
7,Schaeffler Technologies AG &amp; Co. KG,Schaeffler Technologies AG &amp; Co KG,1.0,True,0.997527,1.0,1.0,0.999725
8,"California State University, Fullerton",California State University Fullerton,1.0,True,0.997527,1.0,1.0,0.999725
9,New York Knicks,"New York, Knicks",1.0,True,0.997527,1.0,1.0,0.999725


In [64]:
df_company.head(10)

,in_text,matched,similarity,first_letter_match,dmtph_match,inv_lvst_dist,jw_sim,match_score
0,Valdosta State University,VALDOSTA STATE UNIVERSITY,1.0,True,0.997527,1.0,1.0,0.999725
1,Schaeffler Technologies AG &amp; Co. KG,SCHAEFFLER TECHNOLOGIES AG &amp; CO. KG,1.0,True,0.997527,1.0,1.0,0.999725
2,Homeland Security,homeland security,1.0,True,0.997527,1.0,1.0,0.999725
3,Homeland Security,HOMELAND SECURITY,1.0,True,0.997527,1.0,1.0,0.999725
4,SCHAEFFLER TECHNOLOGIES AG &amp; CO. KG,Schaeffler Technologies AG &amp; Co KG,1.0,True,0.997527,1.0,1.0,0.999725
5,homeland security,HOMELAND SECURITY,1.0,True,0.997527,1.0,1.0,0.999725
6,National Fertilizers Limited,NATIONAL FERTILIZERS LIMITED,1.0,True,0.997527,1.0,1.0,0.999725
7,Schaeffler Technologies AG &amp; Co. KG,Schaeffler Technologies AG &amp; Co KG,1.0,True,0.997527,1.0,1.0,0.999725
8,"California State University, Fullerton",California State University Fullerton,1.0,True,0.997527,1.0,1.0,0.999725
9,New York Knicks,"New York, Knicks",1.0,True,0.997527,1.0,1.0,0.999725


In [65]:
connected_comp_short, connected_comp_long, connected_comp_all = get_postProcessigUnits(connected_comp)

In [66]:
print(len(connected_comp_short))
print(len(connected_comp_long))
print(len(connected_comp_all))

2381
281
2662


In [67]:
connected_comp_short[:20]

[(0, ['1 N. Walnut St.', '113 N. Walnut St.']),
 (1, ['1 S. Main St.', '131 S. Main St.']),
 (2, ['1 The Court', '11 The Court']),
 (3, ['100 Center Street', '1800 Center Street']),
 (4, ['108 1st Ave. South', '108 1st Ave South']),
 (5, ['10th Ave. SE', '10th Ave. South']),
 (6, ['110th Street', '10th Street']),
 (7, ['1140 Lake Ave', '1140 Lake Ave.']),
 (8, ['1146 Villa St.', '1146 Villa Street']),
 (9, ['118 Locust St.', '118 Locust Street']),
 (10, ['12 Curtis St.', '129 Curtis St.']),
 (11, ['1202 South Lynhurst Drive', '1202 S. Lynhurst Drive']),
 (12, ['121 Campion Way', '121 Champion Way']),
 (13, ['1210 State Highway', 'State Highway 121']),
 (14, ['126 3rd St NE', '126 3rd St. NE']),
 (15, ['12th Ave.', '12th Ave']),
 (16, ['13 The Court', '3 The Court']),
 (17, ['1460 Wyoming Avenue', '1460 Wyoming Ave.']),
 (18, ['14th Street North', 'North 14th Street']),
 (19, ['150th Avenue', '50th Avenue'])]

In [68]:
len(connected_comp_long)

281

In [69]:
connected_comp_all

[(0, ['1 N. Walnut St.', '113 N. Walnut St.']),
 (1, ['1 S. Main St.', '131 S. Main St.']),
 (2, ['1 The Court', '11 The Court']),
 (3, ['100 Center Street', '1800 Center Street']),
 (4, ['108 1st Ave. South', '108 1st Ave South']),
 (5, ['10th Ave. SE', '10th Ave. South']),
 (6, ['110th Street', '10th Street']),
 (7, ['1140 Lake Ave', '1140 Lake Ave.']),
 (8, ['1146 Villa St.', '1146 Villa Street']),
 (9, ['118 Locust St.', '118 Locust Street']),
 (10, ['12 Curtis St.', '129 Curtis St.']),
 (11, ['1202 South Lynhurst Drive', '1202 S. Lynhurst Drive']),
 (12, ['121 Campion Way', '121 Champion Way']),
 (13, ['1210 State Highway', 'State Highway 121']),
 (14, ['126 3rd St NE', '126 3rd St. NE']),
 (15, ['12th Ave.', '12th Ave']),
 (16, ['13 The Court', '3 The Court']),
 (17, ['1460 Wyoming Avenue', '1460 Wyoming Ave.']),
 (18, ['14th Street North', 'North 14th Street']),
 (19, ['150th Avenue', '50th Avenue']),
 (20, ['15th District', '5th District']),
 (21, ['15th and 16th streets', '14t

In [81]:
# manual insepction to excude, connect & add
index2exclude = [3,5,79,82,91,299,333]
index2connect = [(10,11,12),(140,141,142),(313,314),(317,319,323,351),(336,337)]
comp2add = ['FULTON','Fulton']

In [82]:
connected_comp_final = get_comp_final(connected_comp_all, connected_comp_short, is_all=True
                                     , index2connect=index2connect
                                     , index2exclude=index2exclude
                                     , comp2add=comp2add
                                     )

In [83]:
print(len(connected_comp))
print(len(connected_comp_final))

352
338


In [84]:
location_connected = connected_comp_final

In [70]:
normalized_terms = choose_normalized(location_connected, choice='long')

In [71]:
raw2normalized, normalized2raw = make_norm_dict(location_connected, normalized_terms)

In [72]:
print("# of raw (in-snippet) entiteis normalized: {}".format(len(raw2normalized)))
print("# of normalized keys: {}".format(len(normalized2raw)))

# of raw (in-snippet) entiteis normalized: 5686
# of normalized keys: 2662


In [73]:
ver_string="_v1"
f_name = 'location_matching' + ver_string
df_location.to_csv("corpus/" + f_name + ".csv")

In [74]:
location_raw2norm = raw2normalized
location_norm2raw = normalized2raw

In [75]:
list(location_raw2norm.items())[:5]

[("Coeur D'Alene, ID", "Coeur d' Alene, ID"),
 ('COEUR D ALENE ID', "Coeur d' Alene, ID"),
 ("Coeur d'Alene, ID", "Coeur d' Alene, ID"),
 ('COEUR D ALENE, ID', "Coeur d' Alene, ID"),
 ("Coeur d'Alene ID", "Coeur d' Alene, ID")]

In [76]:
list(location_norm2raw.items())[:5]

[("Coeur d' Alene, ID",
  ["Coeur D'Alene, ID",
   'COEUR D ALENE ID',
   "Coeur d'Alene, ID",
   'COEUR D ALENE, ID',
   "Coeur d'Alene ID",
   "Coeur d' Alene, ID"]),
 ('San Francisco, CA',
  ['SAN FRANCISCO CA',
   'San Francisco CA',
   'San Francisco, CA.',
   'SAN FRANCISCO, CA',
   'San Francisco, Ca',
   'San Francisco, CA']),
 ('Washington. D.C.',
  ['WASHINGTON, D.C.',
   'Washington, D.C.',
   'WASHINGTON D.C.',
   'Washington D.C.',
   'Washington. D.C.']),
 ('KOOTENAI County, Idaho',
  ['KOOTENAI COUNTY, Idaho',
   'KOOTENAI COUNTY, IDAHO',
   'Kootenai County, Idaho',
   'Kootenai County Idaho',
   'KOOTENAI County, Idaho']),
 ('Los Angeles, California',
  ['Los Angeles CALIFORNIA',
   'LOS ANGELES, California',
   'Los Angeles California',
   'LOS ANGELES CALIFORNIA',
   'Los Angeles, California'])]

In [77]:
data_path="corpus/"

f_name = 'location_raw2norm' + ver_string
json_dp = json.dumps(location_raw2norm)
with open(data_path + f_name + ".json","w") as f:
    f.write(json_dp)

f_name = 'location_norm2raw' + ver_string
json_dp = json.dumps(location_norm2raw)
with open(data_path + f_name + ".json","w") as f:
    f.write(json_dp)